<a href="https://www.kaggle.com/code/mdmosarrofhossen/brain-tumor-detection-accuracy-100-val-acc-100?scriptVersionId=121464837" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Brain Tumor Detection

In this notebook I will include different types of solutions for brain tumor detection. The dataset is collected from [here](https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection). The solutions are collected from different notebooks. As a learning process, I am collecting different solutions from different notebooks, trying to improve performace of the model and comparing the performance of the different model on this dataset.

We will use 3 models on this notebook, which are ResNet50v2, VGG19 and InceptionV3. So the contents of the notebook will be as follows,

* Importing libraries
* Loading Images and creating utility function for data augmentation
* Image demostration
* CNN models
    1. ResNet50V2
        * Training without data augmentation
        * Further tune the model with data augmentation   
    2. VGG19
        * Training without data augmentation
        * Further tune the model with data augmentation 
    3. InceptionV3
        * Training without data augmentation
        * Further tune the model with data augmentation 


Let's get started!

# Importing the libraries


* Numpy and pandas for data analysis
* plotlib and seaborn for data  exploration
* sklearn metrics for measuring the performance of the model
* tensorflow and keras for building our models

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
tf.random.set_seed(42)

# Loading Images and creating utility function for data augmentation


In this section we will load our data(images with labels) from our dataset. The initial size(244*244) will be used for ResNet50v2, we will change the size as required for other models.

We will create two utility function for generating our data to be trained. One is image_gen, which will create train_ds and val_ds without data augmention which will be used for training the model without data augmentation. The other one is augmented_data, which will apply data_augmentation and return augmented data for train_ds which will be used to futher tune our models.

In [ ]:
#intializing the image size and batch size
height = 244
width = 244
batch_size=32

In [ ]:
dataset_dir = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/"

def image_gen(height, width):
    datagen = ImageDataGenerator(
            rescale=1./255.,
            validation_split=0.2,
            )
    train_ds = datagen.flow_from_directory(
        dataset_dir,
        batch_size=batch_size,
        subset="training",
        shuffle=True,
        class_mode="binary",
        target_size=(height, width),
        classes={'no': 0., 'yes': 1.}
    )
    
    val_ds = datagen.flow_from_directory(
        dataset_dir,
        batch_size=batch_size,
        subset="validation",
        shuffle=True,
        class_mode="binary",
        target_size=(height, width),
        classes={'no': 0., 'yes': 1.}
    )
    
    return train_ds, val_ds
    

def augmented_data(height, width):
    datagen = ImageDataGenerator(
        rescale= 1./255.,
        width_shift_range= 0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=30,
        horizontal_flip=True,
        brightness_range=(0.5, 1.0)
    )
    
    train_ds = datagen.flow_from_directory(
        dataset_dir,
        batch_size=batch_size,
        shuffle=True,
        class_mode="binary",
        target_size=(height, width),
        classes={'no': 0., 'yes': 1.}
    )
    
    return train_ds

train_ds, val_ds = image_gen(height, width)

total_images = np.concatenate([train_ds.labels, val_ds.labels])
print('\n\n',{"No brain tumor cases": len(np.where(total_images==0)[0]),
             "Brain tumor cases": len(np.where(total_images==1)[0])})

# Image demostration

Demostrating some of the training data.

In [ ]:
fig, ax = plt.subplots(3,3, figsize=(10,10))
fig.suptitle("Brain Tumor Pictures")

for k in range(9):
    images, labels = train_ds.next()
    #print(images[0].shape)
    i, j = k//3, k%3
    ax[i, j].imshow(images[0])
    ax[i, j].set_title(f"label {int(labels[0])}")
    ax[i, j].axis('off')
plt.show()

# CNN Models

We will start training our data with the CNN models(ResNet50V2, VGG19 and InceptionV2). There will be 2 step for each cases. 

* Training without data augmentation
* Further tune the model with augmented data

# 1. ResNet50V2

We will use ResNet50V2 pre-trained model from keras. You can read more details about it from [here](https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet_v2/ResNet50V2). We will use the pre-trained model as base model and will build a final segement with a GlobalAveragePooling2D layer, Flatten layer, 2 Dense layer and a Dropout layer. We will use softmax activation for our output layer.

**Training without data augmentation**

In [ ]:
from keras.applications import ResNet50V2
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.models import Sequential, load_model

In [ ]:
#Declaring base model
tf.keras.backend.clear_session()

base_model = ResNet50V2(input_shape=(244,244,3), include_top=False)
base_model.trainable = False

model1 = Sequential([
    base_model, 
    GlobalAveragePooling2D(),
    Flatten(),
    Dense(256, activation="relu", kernel_initializer='he_normal'),
    Dropout(0.3),
    Dense(2, activation='softmax')
])

model1.compile(optimizer=Adam(), 
               loss="sparse_categorical_crossentropy", 
               metrics=['accuracy'])

model1.summary()

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model/resnet50v2_best.h5', 
                                                monitor='accuracy', verbose=1, 
                                                mode='max',save_best_only=True)
early = tf.keras.callbacks.EarlyStopping(monitor="accuracy", mode="max",restore_best_weights=True, patience=5)
callbacks_list = [checkpoint,early]

history = model1.fit(train_ds, validation_data=val_ds, epochs=30, callbacks=callbacks_list)

In [ ]:
his_data = pd.DataFrame(history.history)
plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.plot(his_data.loss, label="Training loss")
plt.plot(his_data.val_loss, label="Validation loss")
plt.xlabel("Epochs")
plt.ylabel("loss")
plt.title("Losses")
plt.grid()
plt.legend()

plt.subplot(1,2,2)
plt.plot(his_data.accuracy, label="Training accuracy")
plt.plot(his_data.val_accuracy, label="Validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy")
plt.grid()
plt.legend()

**Further tune the model with augmented data**

We will use our augmented_data utility function to load augmented training data. Here we are using a callback function(reduce_lr) to decrease learning rate if the accuracy doesn't improve for 2 epochs.

In [ ]:
aug_train_ds = augmented_data(height, width)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.3,
                                                 patience=2, min_lr=0.0000001)
callbacks_list = [checkpoint,early,reduce_lr]

history = model1.fit(aug_train_ds, validation_data=val_ds, epochs=30, callbacks=callbacks_list)

In [ ]:
his_data = pd.DataFrame(history.history)
plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.plot(his_data.loss, label="Training loss")
plt.plot(his_data.val_loss, label="Validation loss")
plt.xlabel("Epochs")
plt.ylabel("loss")
plt.title("Losses")
plt.grid()
plt.legend()

plt.subplot(1,2,2)
plt.plot(his_data.accuracy, label="Training accuracy")
plt.plot(his_data.val_accuracy, label="Validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy")
plt.grid()
plt.legend()

In [ ]:
train_result = model1.evaluate(train_ds)
val_result = model1.evaluate(val_ds)

model1_result = pd.DataFrame(zip(train_result, val_result), 
                             columns=['Train', 'Validation'], 
                             index=['Loss', "Accuracy"])

model1_result

In [ ]:
y_pred = model1.predict(val_ds[0][0])
y_pred = np.argmax(y_pred, axis=-1)

y_test = val_ds[0][-1]

#print(y_pred.shape)
#print(y_test.shape)
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("Accuracy Score: ", accuracy_score(y_test,y_pred))
print("Classification report:\n", classification_report(y_pred,y_test))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True)

# 2. VGG19

We will use VGG19 pre-trained model from keras. You can read more details about it from [here](https://keras.io/api/applications/vgg/). We will use the pre-trained model as base model and will build a final segement with a Flatten layer and a Dense layer. We will use sigmoid activation for our output layer. For VGG19, we will use image size of 224*224.

**Training without data augmentation**

In [ ]:
from keras.applications.vgg19 import VGG19

In [ ]:
tf.keras.backend.clear_session()

#initializing new height and width for images
height=224
width=224

train_ds, val_ds = image_gen(height, width)

base_model = VGG19(
    weights='imagenet',
    include_top=False,
    input_shape=(height,width,3)
)

base_model.trainable = False

model2 = Sequential([
    base_model,
    #GlobalAveragePooling2D(),
    Flatten(),
    #Dense(256, activation="relu", kernel_initializer='he_normal'),
    Dense(1, activation='sigmoid')
])

model2.summary()

In [ ]:
model2.compile(loss="binary_crossentropy", 
               optimizer=Adam(0.01), metrics=['accuracy'])

checkpoint = tf.keras.callbacks.ModelCheckpoint('model/vgg19_best.h5', 
                                                monitor='accuracy', verbose=1, 
                                                mode='max',save_best_only=True)
early = tf.keras.callbacks.EarlyStopping(monitor="accuracy", mode="max",
                                         restore_best_weights=True, patience=5)
callbacks_list = [checkpoint,early]

history = model2.fit(train_ds, 
           validation_data=val_ds,
           epochs=30,
           shuffle=True,
           verbose=True,
           callbacks=callbacks_list
          )

In [ ]:
his_data = pd.DataFrame(history.history)
plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.plot(his_data.loss, label="Training loss")
plt.plot(his_data.val_loss, label="Validation loss")
plt.xlabel("Epochs")
plt.ylabel("loss")
plt.title("Losses")
plt.grid()
plt.legend()

plt.subplot(1,2,2)
plt.plot(his_data.accuracy, label="Training accuracy")
plt.plot(his_data.val_accuracy, label="Validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy")
plt.grid()
plt.legend()

**Further tune the model with augmented data**

We will use our augmented_data utility function to load augmented training data. Here we are using a callback function(reduce_lr) to decrease learning rate if the accuracy doesn't improve for 2 epochs.

In [ ]:
aug_train_ds = augmented_data(height, width)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.3,patience=2, 
                                                 min_lr=0.0000001)
callbacks_list = [checkpoint,early, reduce_lr]

history = model2.fit(aug_train_ds, 
           validation_data=val_ds,
           epochs=30,
           shuffle=True,
           verbose=True,
           callbacks=callbacks_list
          )

In [ ]:
his_data = pd.DataFrame(history.history)
plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.plot(his_data.loss, label="Training loss")
plt.plot(his_data.val_loss, label="Validation loss")
plt.xlabel("Epochs")
plt.ylabel("loss")
plt.title("Losses")
plt.grid()
plt.legend()

plt.subplot(1,2,2)
plt.plot(his_data.accuracy, label="Training accuracy")
plt.plot(his_data.val_accuracy, label="Validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy")
plt.grid()
plt.legend()

In [ ]:
train_result = model2.evaluate(train_ds)
val_result = model2.evaluate(val_ds)

model2_result = pd.DataFrame(zip(train_result, val_result), 
                             columns=['Train', 'Validation'], 
                             index=['Loss', "Accuracy"])

model2_result

In [ ]:
y_pred = model2.predict(val_ds[0][0])
y_pred = np.array([1 if x > 0.5 else 0 for x in y_pred])

y_test = val_ds[0][-1]

#print(y_pred.shape)
#print(y_test.shape)
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("Accuracy Score: ", accuracy_score(y_test,y_pred))
print("Classification report:\n", classification_report(y_pred,y_test))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True)

# 3. InceptionV3

We will use InceptionV3 pre-trained model from keras. You can read more details about it from [here](https://keras.io/api/applications/inceptionv3/). We will use the pre-trained model as base model and will build a final segement with a Flatten layer and a Dense layer. We will use sigmoid activation for our output layer.

**Training without data augmentation**

In [ ]:
from keras.applications import InceptionV3

In [ ]:
height=299
width=299

train_ds, val_ds = image_gen(height, width)

tf.keras.backend.clear_session()

base_model = InceptionV3(
    input_shape=(height, width,3),
    weights='imagenet',
    include_top=False
)

base_model.trainable = False

model3 = Sequential([
    base_model,
    Flatten(),
    Dense(1, activation='sigmoid')
])

model3.compile(loss="binary_crossentropy", optimizer=Adam(0.01), metrics=['accuracy'])

model3.summary()

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model/inceptionv3_best.h5', 
                                                monitor='accuracy', verbose=1, 
                                                mode='max',save_best_only=True)
early = tf.keras.callbacks.EarlyStopping(monitor="accuracy", mode="max",
                                         restore_best_weights=True, patience=5)

callback_lsit = [checkpoint, early]

history = model3.fit(train_ds, validation_data=val_ds, epochs=30, callbacks=callback_lsit)

In [ ]:
his_data = pd.DataFrame(history.history)
plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.plot(his_data.loss, label="Training loss")
plt.plot(his_data.val_loss, label="Validation loss")
plt.xlabel("Epochs")
plt.ylabel("loss")
plt.title("Losses")
plt.grid()
plt.legend()

plt.subplot(1,2,2)
plt.plot(his_data.accuracy, label="Training accuracy")
plt.plot(his_data.val_accuracy, label="Validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy")
plt.grid()
plt.legend()

**Further tune the model with augmented data**

We will use our augmented_data utility function to load augmented training data. Here we are using a callback function(reduce_lr) to decrease learning rate if the accuracy doesn't improve for 3 epochs.

In [ ]:
aug_train_ds = augmented_data(height, width)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.3,
                              patience=3, min_lr=0.0000001)
callback_lsit = [checkpoint, early, reduce_lr]

history = model3.fit(aug_train_ds, validation_data=val_ds, epochs=30, callbacks=callback_lsit)

In [ ]:
his_data = pd.DataFrame(history.history)
plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.plot(his_data.loss, label="Training loss")
plt.plot(his_data.val_loss, label="Validation loss")
plt.xlabel("Epochs")
plt.ylabel("loss")
plt.title("Losses")
plt.grid()
plt.legend()

plt.subplot(1,2,2)
plt.plot(his_data.accuracy, label="Training accuracy")
plt.plot(his_data.val_accuracy, label="Validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy")
plt.grid()
plt.legend()

In [ ]:
train_result = model3.evaluate(train_ds)
val_result = model3.evaluate(val_ds)

model3_result = pd.DataFrame(zip(train_result, val_result), 
                             columns=['Train', 'Validation'], 
                             index=['Loss', "Accuracy"])

model3_result

In [ ]:
y_pred = model3.predict(val_ds[0][0])
y_pred = np.array([1 if x > 0.5 else 0 for x in y_pred])

y_test = val_ds[0][-1]

#print(y_pred.shape)
#print(y_test.shape)
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("Accuracy Score: ", accuracy_score(y_test,y_pred))
print("Classification report:\n", classification_report(y_pred,y_test))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True)

# Conclusion

We have a great results on our all three models. With data augmentation, we have increased the validation accuracy(val_accuray) significatly also reduced over-fitting to the training dataset. Since we had a really small dataset, the models might overfit to the train and validation data which can be reduced by adding more data. Overall we have achieved a great results.

A little comparison between the three models:


                    train_Accuracy   val_accuracy    f1_score

1. ResNet50V2:
                        
                         99%            ~96%          0.97
2. VGG19:

                        100%           ~100%          1.00                        
3. InceptionV3:

                        100%           ~98%          1.00

Please upvote if you like this notebook. Thank you!